In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Must import comet before torch, sometimes throws error otherwise.
from img_wang.callbacks import CometCallbackWithGrads
from fastai2.vision.all import show_image, show_images
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, f1_score, recall_score, \
                            precision_score, roc_auc_score
import torch
import torch.nn as nn
import torch.nn.functional as F

from htools import *
from img_wang.data import load_img, get_databunch
from img_wang.models import Encoder, SingleInputBinaryModel, \
    TorchvisionEncoder
from img_wang.torch_utils import gpu_setup, top_mistakes
from incendio.callbacks import MetricHistory, ModelCheckpoint, EarlyStopper
from incendio.core import Trainer
from incendio.metrics import mean_soft_prediction, std_soft_prediction, percent_positive

In [5]:
cd_root()
gpu_setup()

Current directory: /storage/img-wang
Setting seeds for reproducible training.


## Load v0 model (stopped saving weights after v1)

In [6]:
dst, dsv, dlt, dlv = get_databunch('data/imagewang-160/unsup/', 
                                   mode='patchwork', bs=256, pct_pos=.5)

In [7]:
net = SingleInputBinaryModel(TorchvisionEncoder(), 
                             nn.Linear(2560, 1))

In [8]:
t = Trainer(net, dst, dsv, dlt, dlv, F.binary_cross_entropy_with_logits,
            mode='binary', out_dir='data/models/v0', last_act=torch.sigmoid,
            callbacks=[MetricHistory(None, None),
                       CometCallbackWithGrads('img_wang'),
                       EarlyStopper('loss', 'min', patience=5)],
            metrics=[mean_soft_prediction, std_soft_prediction, percent_positive, 
                     accuracy_score, f1_score, recall_score, precision_score,
                     roc_auc_score])

In [9]:
t.load('trainer.pkl')

2020-09-15 05:53:57,388 [INFO]: Loading weights from data/models/v0/trainer.pkl.


In [23]:
xb, yb = next(iter(dlt))
dft = top_mistakes(t, xb, yb)

AttributeError: 'Tensor' object has no attribute 'idx'

In [15]:
dft

,y,y_proba
0,0.0,0.021437
1,1.0,0.939200
2,0.0,0.726055
3,1.0,0.968426
4,0.0,0.020999
...,...,...
251,0.0,0.009739
252,1.0,0.970678
253,1.0,0.991280
254,0.0,0.089755


Some of these samples are labeled as negatives but definitely look like positives. I wonder if there are duplicates in the dataset? If not, that would be a weird bug - can't imagine what would cause that.

There was at least 1 example, though, that looked like the opposite problem (labeled as positive but looks like it came from a different image). That seems like evidence in favor of the "bug" issue over the "duplicates" issue.